### This notebook defines test cases ensuring the DB fits the busyness neeeds

In [1]:
import pandas as pd 

from pgnotebook import PostgreSQL

pg = PostgreSQL(port=8000)

#### Using create script generated from schema (http://dbdesigner.net/designer/schema/93018)

In [2]:
pg.sql('drop schema if exists rail cascade')
with open('create.sql') as f:
    pg.sql(f.read())
with open('populate.sql') as f:
    pg.sql(f.read())

In [3]:
pg.show_tables(schema='rail')  # 21 tables created, the script also generated all the necessary constraints

,schemaname,tablename
0,rail,user
1,rail,card
2,rail,transaction_status
3,rail,transaction
4,rail,passenger
5,rail,reservation
6,rail,train_type
7,rail,car_type
8,rail,car
9,rail,seat_type


#### The core functionality to test:

1. user must be able to query all routes / specific runs that fit desired (date, origin, destination) values
2. user must be able to query all available seats for a specific run (and therefore route)

In [4]:
pg.sql('select * from rail.route')  # we have 2 routes stored in DB

,id,type_id
0,1,1
1,2,2


In [5]:
pg.sql("""
select * 
from rail.route_station 
order by route_id, station_index
""")  # this is how the route map looks like

,route_id,station_id,station_index,time_arrival,time_departure
0,1,1,1,09:19:43.237876,09:19:43.237876
1,1,2,2,09:19:43.237876,09:19:43.237876
2,1,3,3,09:19:43.237876,09:19:43.237876
3,1,4,4,09:19:43.237876,09:19:43.237876
4,1,5,5,09:19:43.237876,09:19:43.237876
5,1,7,6,09:19:43.237876,09:19:43.237876
6,2,1,1,09:19:43.237876,09:19:43.237876
7,2,2,2,09:19:43.237876,09:19:43.237876
8,2,3,3,09:19:43.237876,09:19:43.237876


### Case: Search for all routes that fit desired origin and destination

In [6]:
# assuming we've given origin_id = 1 and destination_id = 4
def get_routes(origin, destination):
    return pg.sql("""
    SELECT o.route_id
    FROM
    (
      SELECT route_id, station_index
      FROM rail.route_station
      WHERE station_id = {}
    ) o
    LEFT JOIN
    (
      SELECT route_id, station_index
      FROM rail.route_station
      WHERE station_id = {}
    ) d
      ON o.route_id = d.route_id
    WHERE
      o.station_index < d.station_index
      AND
      o.route_id = d.route_id
    """.format(origin, destination))

In [7]:
get_routes(1, 3)  # 2 routes available

,route_id
0,1
1,2


In [8]:
get_routes(1, 4)  # should be only 1 route available

,route_id
0,1


In [9]:
get_routes(3, 1)  # no routes in opposite direction

""


### Case: Search for all runs(scheduled voyages) that fit desired origin and destination
- Similiar to the previous example, but we are looking for runs with specific date and route

In [10]:
# is also possible to pack this in one sql query
# here we'll reuse previous example for simplicity
def get_runs(origin, destination, date):

    df = get_routes(origin, destination)
    if df.empty:
        return pd.DataFrame()
    else:
        routes = df.route_id.tolist()
    query_routes_arg = ', '.join(map(str, routes))
    return pg.sql("""
    select * from run 
    where route_id in ({}) and date = '{}'
    """.format(query_routes_arg, date))

In [11]:
get_runs(origin=1, destination=3, date='2017-06-01')

,base_rate,date,id,route_id,train_id
0,100,2017-06-01,2,1,1
1,150,2017-06-01,3,2,1


In [12]:
get_runs(origin=1, destination=3, date='2017-05-31')

,base_rate,date,id,route_id,train_id
0,100,2017-05-31,1,1,1


In [13]:
get_runs(origin=1, destination=4, date='2017-06-01')

,base_rate,date,id,route_id,train_id
0,100,2017-06-01,2,1,1


In [14]:
get_runs(origin=4, destination=1, date='2017-06-01')

""


### Case check available tickets for a specific run

### Query logic
In scope of specific run compare the set of available seats in the train with the set of booked tickets
- if seat has no booked tickets attached to it
 - it's available
- else check the difference of two sets:
 - set(A) contains all station indexes that booked for the seat on the run
 - set(B) contains all station indexes that desired to be booked
 
The following example demonstrates the algorithm using python types though it probably can be implemented in pure SQL as well

In [15]:
# exclude bording and get-off stations from booked ticket indexes
a = {1, 2}
b = {4, 5, 6, 7}

# this is desired route to book a ticket for
c = {2, 3, 4}

In [16]:
a.remove(max(a)) 
a.remove(min(a))

b.remove(max(b))
b.remove(min(b))

occupied = a.union(b)
occupied  # this station_indexes for the seat is occupied

{5, 6}

##### If there's no intersection between occupied set and desired set then the seat can be actualy booked 

In [18]:
# now to determine if we can book desired ticket
assert c.intersection(occupied) == set()  # no intersection